<a href="https://colab.research.google.com/github/rsnatorres/analise_dados_ramon/blob/main/crawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Improving Tasks

1. VPN switcher still malfunctioning: not always connecting to random choice


Using Cached google Data

http://webcache.googleusercontent.com/search?q=cache:URL(URL of the website you want to scrape).

# Importing and config

## Packages

In [ ]:
# get html
from bs4 import BeautifulSoup as soup
# simulating human interaction
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
# as duas importações a baixo são para lidar com o stop no meio do crawler
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
# third party webdriver manager --> so dont need to download webdriver everytime
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
# handle  swichting servers via nordvpn
import subprocess
from subprocess import check_output, DEVNULL
import psutil
import json
import requests
# other
import os.path # to know if a file already exists --> append or write file
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import re # regular expressions
import random
import time
import os
from scipy.stats import powerlaw

## Configs

In [ ]:
ignored_exceptions = (NoSuchElementException,StaleElementReferenceException)

In [ ]:
cwd = os.getcwd()
cwd

'D:\\OneDrive\\0 - Torres Consultoria\\projetos\\Crawlers projects'

# Functions

## Conn with usable IP

In [ ]:
# VPN Funcion
# sample code based on nordvpn switcher (discontinued package)
#https://github.com/kboghe/NordVPN-switcher/blob/master/nordvpn_switcher/nordvpn_switch.py
# Switching VPNs
def get_nordvpn_servers():
    print('Getting NordVpn Servers')
    serverlist =  soup(requests.get("https://nordvpn.com/api/server").content,"html.parser")
    site_json = json.loads(serverlist.text)

    global filtered_servers_win
    filtered_servers_win = [] #windows
    for specific_dict in site_json:
        try:
            if specific_dict['categories'][0]['name'] == 'Standard VPN servers':
                filtered_servers_win.append(specific_dict['name'])
            else:
                pass
        except IndexError:
            pass
    return filtered_servers_win

def connect_nordvpn_server(cwd_path = 'C:/Program Files/NordVPN', country = 'Brazil'):
    print('Establishing NordVPN Connection')
    try:
        country_servers = [x for x in filtered_servers_win if country in x]
    except:
        country_servers = [x for x in get_nordvpn_servers() if country in x]
    global random_choice_server
    random_choice_server = random.choice(country_servers)
    print(f"Connecting to '{random_choice_server}' server")
    open_nord_win = subprocess.Popen(
        ["nordvpn", "-c", "-n", random_choice_server],
         shell=True, cwd=cwd_path,stdout=DEVNULL)
    return open_nord_win

# connect_nordvpn_server() must run twice to work properly
def try_nordvpn_connection_until_connected():
    #trying_list = []
    try:
        connect_nordvpn_server()
        print('Waiting for NordVPN app to open')
        time.sleep(20)
        try:
            connect_nordvpn_server() # break point: server connection successful
        except:
            try_nordvpn_connection_until_connected()
            print(f'Trying open nordvpn again')
    except:
        print(f'Trying open nordvpn again')
        try_nordvpn_connection_until_connected()
    print(f'Connection established')

def request_until_get_ip():
    try:
        global ip
        ip = requests.get('https://api.ipify.org').text
    except:
        print('requesting ip again...')
        request_until_get_ip()

def try_nordvpn_conn_usable_ip(days_blocked = 180):
    # 1. connects with nordvpn
    try_nordvpn_connection_until_connected()
    print('Waitting until Ip request')
    time.sleep(20) # time for ip change?
    print('Getting IP')
    #global ip
    #ip = get_external_ip()
    # ip = requests.get('https://api.ipify.org').text
    request_until_get_ip()
    global df_blocked_ips
    if os.path.exists(FILE_BLOCKED_IPS): # file
        df_blocked_ips = pd.read_csv(FILE_BLOCKED_IPS,
                                 sep = ';',
                                 parse_dates=['date_time'])
        six_months_ago = datetime.now() - timedelta(days_blocked)
        blocked_ips = df_blocked_ips[df_blocked_ips['date_time'] >= six_months_ago]['ip'].unique()

        if ip in blocked_ips:
            print(f'{ip} was recently blocked in {URL}')
            try_nordvpn_conn_usable_ip()
    else:
        pass

After connection with URL

In [ ]:
def update_ips_block_list(file_blocked_ips = 'file_blocked_ips.csv'):
    if os.path.isfile(file_blocked_ips):
        f = open(file_blocked_ips, "a", encoding='utf-8\n')
        f.write(TODAY + ';' + ip + ';' + random_choice_server + '\n')
        f.close()
        print(f'Ip {ip} inserted into blocked_file')
    else:
        f = open(file_blocked_ips, "w", encoding='utf-8\n')
        f.write('date_time' + ';' + 'ip' + ';' + 'server' + '\n')
        f.write(TODAY + ';' + ip + '\n')
        f.close()
        print(f'Creating blocked_files: Ip {ip} inserted into blocked_file')

def check_if_ip_blocked():
    # checking if ip is blocked in URL page
    if "Sorry, you have been blocked" in driver.page_source:
        update_ips_block_list(file_blocked_ips = 'file_blocked_ips.csv')
        return True
    else:
        print(f'{ip} is not blocked and request to URL was successful')
        return False

In [ ]:
# final function
def try_url_conn_with_usable_ip(url,
                                trying_list = [],
                                days_blocked = 180):
        # connects with a usable ip BASED ON HISTORIC BLOCKED IPS
    try_nordvpn_conn_usable_ip(days_blocked = 180)
        # connect with URL
    global driver
    driver = webdriver.Chrome(
        service=ChromeService(ChromeDriverManager().install()))
    driver.get(url)
    time.sleep(5)
    if check_if_ip_blocked() == True:
        driver.quit()
        print('current ip is blocked, quitting driver and start again')
        try_url_conn_with_usable_ip()
    driver.maximize_window()
    print(f'Connection Successful with ip: "{ip}" & server: "{random_choice_server}"')

In [ ]:
def try_url(url):
    global driver
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
    driver.get(url)
    driver.maximize_window()
    time.sleep(5)

In [ ]:
#try_url(url = URL)

## Mimic users behaviour

Functions that mimics users behaviour

In [ ]:
# Interaction Functions (webdriver/selenium functions)
def find_element_by_xpath(xpath):
    return WebDriverWait(driver, 10, ignored_exceptions=ignored_exceptions).until(
                            EC.presence_of_element_located((By.XPATH, xpath)))
def find_element_by_css(css_selector):
    return WebDriverWait(driver, 10, ignored_exceptions=ignored_exceptions).until(
                            EC.presence_of_element_located((By.CSS_SELECTOR, css_selector)))
def find_all_elements(xpath):
    return WebDriverWait(driver, 10, ignored_exceptions=ignored_exceptions).until(
                            EC.presence_of_all_elements_located((By.XPATH,  xpath)))
# robust alternative if element.click() fails
def click_button(button):
    time.sleep(np.random.randint(2,4))
    return driver.execute_script("arguments[0].click();", button)

def scroll_until_visible(button):
    current_position = driver.execute_script("return window.scrollY")
    target_position = button.location['y']

    while current_position < target_position:
        step = 70

        driver.execute_script(f"window.scrollBy(0, {step});")
        current_position += step
        time.sleep(powerlaw.rvs(0.07, # the higher --> less concentration in corner
                   scale = 2.5, # maximum value -> 3 seconds
                   size = 1)[0])

## Site Crawling

## Setting Ambient Vars

### Home Search

Mostly sites we must specify with 3 things:
1. COMPRAR_OU_ALUGAR
2. TIPO_IMOVEL
3. city

COMPRAR_OU_ALUGAR

In [ ]:
# Find and click option based on condition
#COMPRAR_OU_ALUGAR = "'Comprar'" # double so can enter in .format() like 'word'

In [ ]:
def define_buy_or_rent(comprar_ou_alugar = 'Comprar'):
    button_comprar_ou_alugar = find_element_by_xpath(
                f"//button[text() = '{comprar_ou_alugar}']")
    #button_comprar_ou_alugar.text
    click_button(button_comprar_ou_alugar)

TIPO_IMOVEL

In [ ]:
def define_real_estate_tipology_and_use(residencial_e_ou_comercial = ['Residencial'],
                                        tipo_imovel = ['Apartamento']):
    # find and click dropdown botton --> Tipos de Imóveis
    dropdown_button = find_element_by_xpath(
                    "//button[contains(descendant::*/text(), 'Todos os imóveis')]")
    click_button(dropdown_button)
    # select box options
    for finalidade in residencial_e_ou_comercial:
        for tipo in tipo_imovel:
            time.sleep(np.random.randint(1,3))
            #print(finalidade, tipo)
            try:
                option_dropdown_button = find_element_by_xpath(
                f"""//div[@class='l-multiselect__item']
                    //p[contains(text(), '{finalidade}')]
                    /..
                    //div[@class='l-multiselect__input']
                    //*[text() = '{tipo}']""")
                scroll_until_visible(option_dropdown_button)
                time.sleep(np.random.randint(1,3))
                click_button(option_dropdown_button)
            except:
                pass

city

In [ ]:
def define_locality(city = 'Belo Horizonte', bairro = None):
    """city must be a city!!!"""
    # dropdown button
    dropdown_button = find_element_by_xpath("//input[@placeholder= 'Digite o nome da rua, bairro ou cidade']")
    scroll_until_visible(dropdown_button)
    click_button(dropdown_button)
    # write and click options
    try:
        if bairro == None:
            locality = city
            for letter in locality:
                time.sleep(random.randint(0,1))
                dropdown_button.send_keys(letter)
            time.sleep(3)
            # mark options
            option_dropdown_button = find_element_by_xpath(
                        f"""//div[@data-cy='locations-item']
                            //p[contains(text(), 'Cidades')]
                            /..
                            //div[@class='l-multiselect__input']
                            //*[contains(translate(text(), 'ABCDEFGHIJKLMNOPQRSTUVWXYZ', 'abcdefghijklmnopqrstuvwxyz'),
                                                    '{city.lower()}')]
                            /..""")
            click_button(option_dropdown_button)
        else:
            locality = city + ' ' + bairro
            for letter in locality:
                time.sleep(random.randint(0,1))
                dropdown_button.send_keys(letter)
            time.sleep(3)
            # mark options
            option_dropdown_button = find_element_by_xpath(
                f"""//div[@data-cy='locations-item']
                    //p[contains(text(), 'Bairros')]
                    /..
                    //div[@class='l-multiselect__input']
                    //*[contains(translate(text(), 'ABCDEFGHIJKLMNOPQRSTUVWXYZ', 'abcdefghijklmnopqrstuvwxyz'),
                                '{city.lower()}')
                    and contains(translate(text(), 'ABCDEFGHIJKLMNOPQRSTUVWXYZ', 'abcdefghijklmnopqrstuvwxyz'),
                                '{bairro.lower()}')]
                    /..""")
            click_button(option_dropdown_button)
    except:
        time.sleep(10)
        for _ in len(locality):
            dropdown_button.send_keys(Keys.BACKSPACE)
        define_locality(city = city, bairro = bairro)

SUBMIT

In [ ]:
def submit_search():
    button_submit_search = find_element_by_xpath("//button[@type= 'submit']")
    click_button(button_submit_search)

Home Search (FINAL)

In [ ]:
def home_search(comprar_ou_alugar = 'Comprar',
                residencial_e_ou_comercial = ['Residencial'],
                tipo_imovel = ['Apartamento'],
                city = 'Belo Horizonte',
                bairro = None):

    define_buy_or_rent(comprar_ou_alugar = comprar_ou_alugar)
    define_real_estate_tipology_and_use(residencial_e_ou_comercial = residencial_e_ou_comercial,
                                        tipo_imovel = tipo_imovel)
    define_locality(city = city, bairro = bairro)
    submit_search()

## Open Csv

In [ ]:
def open_csv(FILE_NAME, mode = "w"):# if a csv file already exists
    global f
    if mode == "a":#os.path.isfile(FILE_NAME):
        try:
            global last_page_number
            last_page_number = int(pd.read_csv(FILE_NAME, sep=';')['page'].max()) # last page mined
        except Exception as e:
            print(f"Error: {e}")
        finally:
            f = open(FILE_NAME, "a", encoding='utf-8\n') # APPEND
    elif mode == "w":
        f = open(FILE_NAME, "w", encoding='utf-8\n') # WRITE a NEW file
        headers = """TODAY;COMPRAR_OU_ALUGAR;RESIDENCIAL_E_OU_COMERCIAL;TIPO_IMOVEL;city;title;address;area;rooms;bathroom;garag;price;condo_price;amenities;id_;href;page;\n"""
        f.write(headers)
    else:
        return "Error in conditions"

## Feature Extraction

In [ ]:
#search_results = find_element_by_xpath("//div[@class= 'results-list js-results-list']")
#n_search_results = find_element_by_xpath("//strong[@class= 'results-summary__count js-total-records']")

In [ ]:
# ALL PAGES: results quantity for search scattered along
#n_search_results.text

In [ ]:
# getting html
def get_containers_in_html_by_div_xpath(div_xpath = "//div[@class= 'results-list js-results-list']"):
    search_results = find_element_by_xpath(div_xpath)
    html_content = search_results.get_attribute('outerHTML')
    page_soup = soup(html_content, "html.parser")
    containers = page_soup.findAll('article', 'class' == 'property-card__container js-property-card')
    global n_containers
    n_containers = len(containers)
    #len(containers) # len of containers should always match len of cards in each page
    # we should always use the xpath that leads to this result
    return containers

In [ ]:
# specific function for extration of text features
def feature_text_extraction(xpath, container):
    # transforma xpath to specific input formats needed by bsp find()
    combined_pattern = re.compile(r"//(\w+)\[@([^']+) = '([^']+)'")

    tag            = combined_pattern.match(xpath).group(1)
    attribute_name = combined_pattern.match(xpath).group(2)
    atribute_value = combined_pattern.match(xpath).group(3)

    try:
        text = container.find(tag, {attribute_name : atribute_value}).get_text().strip()
        text = re.sub(r'\s+', ' ', text)
        return text
    except:
        return np.nan # f.write('Nan' + ';')

In [ ]:
def attribute_text_extraction(xpath, attribute, data_index):
     try:
        text = find_element_by_xpath(xpath).get_attribute(attribute)
        return text
     except:
        return np.nan

In [ ]:
def get_and_write_page_features_for_each_container(containers):
    page = find_element_by_css('button.js-change-page[data-active]').text
    for i, container in enumerate(containers):
        # features using soup
        title       = feature_text_extraction("//span[@class = 'property-card__title js-cardLink js-card-title']",
                                        container = container)
        address     = feature_text_extraction("//span[@class = 'property-card__address']",
                                        container = container)
        area        = feature_text_extraction("//span[@class = 'property-card__detail-value js-property-card-value property-card__detail-area js-property-card-detail-area']",
                                        container = container)
        rooms       = feature_text_extraction("//span[@class = 'property-card__detail-value js-property-card-value']",
                                        container = container)
        bathroom    = feature_text_extraction("//span[@class = 'property-card__detail-value js-property-card-value']",
                                        container = container)
        garage      = feature_text_extraction("//span[@class = 'property-card__detail-value js-property-card-value']",
                                        container = container)
        price       = feature_text_extraction("//div[@class = 'property-card__price js-property-card-prices js-property-card__price-small']",
                                        container = container)
        condo_price = feature_text_extraction("//strong[@class = 'js-condo-price']",
                                        container = container)
        amenities   = '/'.join(i.text.strip() for i in container.findAll('li', {'class': 'amenities__item'}))
        # using selenium --> these are not texts but attributes of html
        id_    = attribute_text_extraction(
                    #f"""//div[@data-type='property' and @data-index='{i}']""",
                    f"""//div[@id and (@data-type ='property' or @data-index = '{i}')]""",
                    data_index = i,
                    attribute = 'id')
        href   = attribute_text_extraction(
                    #f"""//div[@data-type='property' and @data-index='{i}']//a""",
                   f"""//div[@id and (@data-type ='property' or @data-index = '{i}')]//a""",
                   data_index = i,
                   attribute = 'href')
        # storing AMBIENT VAR for log purposes
        today = TODAY
        #comprar_ou_alugar = COMPRAR_OU_ALUGAR
        residencial_e_ou_comercial = RESIDENCIAL_E_OU_COMERCIAL
        tipo_imovel = TIPO_IMOVEL
        #city = city
        # gettin all together for writing
        variables = [today, comprar_ou_alugar, residencial_e_ou_comercial, tipo_imovel,
                     city, title, address, area, rooms, bathroom, garage, price,
                     condo_price, amenities, id_, href, page]
        for var in variables:
            f.write(str(var) + ';')
        f.write('\n')

# Crawl

## Ambient Vars

In [ ]:
df_bairros_bh = pd.read_csv('df_bairros_com_mais_imoveis_bh.csv')
array_bairros_com_mais_imoveis_bh = df_bairros_bh.sort_values(
    by='ano', ascending=False)['bairro'].str.lower().unique()

In [ ]:
#USUARIO = ''
#SENHA = ''
URL = 'https://www.vivareal.com.br/'
FILE_BLOCKED_IPS = 'file_blocked_ips_viva_real.csv' # columns: ['TODAY', 'blocked_ip', 'blocked_server']
TODAY = pd.to_datetime('today') # Is this an ambient var?

CITIES = ['Contagem', 'Lagoa Santa'] # for more cities, filename should be changed
#BAIRROS = ['Boa Vista', 'Santa Inês', 'São Geraldo', 'Horto', 'Sagrada Família']
COMPRAR_OU_ALUGAR = ["'Comprar'"]
RESIDENCIAL_E_OU_COMERCIAL = ['Residencial']
TIPO_IMOVEL = ['Apartamento', 'Casa']

## Run

The part which contains the architecture and STRATEGY of the webscraping

How we activate each function, where and when

In [ ]:
# test
CITY = 'Vespasiano'
opcao = 'Comprar'
try_url(url = URL)

In [ ]:
home_search(comprar_ou_alugar = opcao,
                city = CITY,
                #bairro = bairro,
                residencial_e_ou_comercial = RESIDENCIAL_E_OU_COMERCIAL,
                tipo_imovel = TIPO_IMOVEL)

In [ ]:
for opcao in COMPRAR_OU_ALUGAR:
    for CITY in CITIES:
        FILE_NAME = f'viva_real_{CITY}_{TODAY.strftime("%m-%d-%Y")}.csv'
        #for bairro in BAIRROS:
        # open file
        #try:
        open_csv(FILE_NAME, #mode='w'
                ) # open csv_file
        # enter site with an unblocked ip
        #try_url_conn_with_usable_ip(url = URL)
        try_url(url = URL) # for fast & manual vpn connection
        # fill home search boxes with ambient vars and submit search
        home_search(comprar_ou_alugar = opcao,
                    city = CITY,
                    #bairro = bairro,
                    residencial_e_ou_comercial = RESIDENCIAL_E_OU_COMERCIAL,
                    tipo_imovel = TIPO_IMOVEL)
        time.sleep(2) # wait for results page to load
        # SubSearch
        # get the number of results (containers or cards)
        n_search_results = int(
                        find_element_by_xpath(
                            "//strong[@class= 'results-summary__count js-total-records']").text.replace('.',''))
        n = 0
        while n < n_search_results:
            time.sleep(2)
            # scroll all page
            button_next_page = find_element_by_xpath("//button[@title = 'Próxima página']")
            #scroll_until_visible(button_next_page)
            # get containers
            containers = get_containers_in_html_by_div_xpath(
                            div_xpath = "//div[@class= 'results-list js-results-list']")
            # write containers into csv
            get_and_write_page_features_for_each_container(containers = containers)
            # go to next page
            click_button(button_next_page)
            n += n_containers # global var
            print(f"{opcao}: {CITY}: {bairro}: {n} results mined")
        f.close()
        driver.quit()
        #except Exception as e:
        #    print(f"ERROR: {e}")
        #    f.close()
        #    break
            #drive.quit() # let it be open for browse investigation for a while

In [ ]:
#driver.quit()

In [ ]:
button_next_page = find_element_by_xpath("//button[@title = 'Próxima página']")

In [ ]:
click_button(button_next_page)

# Analysis

In [ ]:
df = pd.read_csv(FILE_NAME, sep=';')

In [ ]:
len(df), df['id_'].nunique()

In [ ]:
df